In [1]:
# Initial Imports
import numpy as np
import time
import sys

In [2]:
# Creating card deck and scoring rules
cards = ["BONUS: 300"]*12 + ["BONUS: 400"]*10 + ["BONUS: 500"]*8 + ["NO DICE"]*8 + ["FILL 1000"]*6 + ["MUST BUST"]*4 + ["VENGEANCE 2500"]*4 + ["DOUBLE TROUBLE"]*2
scoring_dict = {(5,): 50, (1,): 100, (2,2,2): 200, (3,3,3): 300, (4,4,4): 400, (5,5,5): 500, (6,6,6): 600, (1,1,1): 1000, (1,2,3,4,5,6): 1500}

In [3]:
# Defining class for player
class Player:
    # Instantiating base state
    def __init__(self, name):
        self.name = name
        self.card = ""
        self.dice = []
        self.saved_dice = []
        self.turn_score = 0
        self.score = 0
        
    def new_card(self, deck):
        self.saved_dice = []
        self.card = deck.draw()
        self.dice = [0,0,0,0,0,0]
    
    def roll_dice(self):
        self.dice = np.random.randint(low=1, high=7, size=len(self.dice)).tolist()
        self.saved_dice.clear()
        print(self.dice)
        return self.score_dice(self.dice.copy())
    
    def score_dice(self, to_score):
        possible_score = 0
        dice_counts = dict()
        for num in range(1,7):
            dice_counts[num] = to_score.count(num)
        if set(dice_counts.values()) == {1}:
#            print("Straight! 1500 points")
            to_score.clear()
            return 1500
        for key in dice_counts.keys():
            if dice_counts[key] == 6:
#                print(f"Two sets of {key}: {2*scoring_dict[(key, key, key)]} points")
                to_score.clear()
                return 2*scoring_dict[(key, key, key)]
            if dice_counts[key] > 2:
#                print(f"Set of {key}: {scoring_dict[(key, key, key)]} points")
                possible_score += scoring_dict[(key, key, key)]
                dice_counts[key] -= 3
                for i in range(3):
                    to_score.remove(key)
            if (key in [1, 5]) and (dice_counts[key] > 0):
#                print(f"{dice_counts[key]} {key}s: {scoring_dict[(key,)]} points each (up to {dice_counts[key]*scoring_dict[(key,)]})")
                possible_score += dice_counts[key]*scoring_dict[(key,)]
                for i in range(dice_counts[key]):
                    to_score.remove(key)
        if possible_score == 0:
            print("Bust!")
        return possible_score
    
    def save_dice(self):
        while True:
            scoring_set = input("Which dice would you like to save?")
            try:
                if len(scoring_set) == 1:
                    scoring_set = [int(scoring_set)]
                else:
                    scoring_set = scoring_set.split(",")
                    scoring_set = [int(entry.strip()) for entry in scoring_set]
                break
            except:
                print("Unable to parse - please enter a scoring set of comma-separated dice values")
            try:
                for die in scoring_set:
                    self.dice.remove(die)
                    self.saved_dice.append(die)
                self.turn_score += score_dice(self.saved_dice)
                if saved_dice:
                    raise
            except:
                print("Your input was not in dice or non-scoring dice were present")
                
    def end_turn(self):
        if self.saved_dice == []:
            print("You can't end your turn until you've chosen some valid scoring dice to save")
        else:
            self.score += self.turn_score
            
    # Method to quit the game
    def quit_game(self):
        #if input("Are you sure you want to quit? (y/n)").lower().strip() == "y":
        sys.exit("Thank you for playing!")

In [4]:
# Deck class to store cards
class Deck:
    def __init__(self):
        self.draw_pile = cards.copy()
        np.random.shuffle(self.draw_pile)
    
    def draw(self):
        if len(self.draw_pile) < 1:
            print("New Deck!")
            self.draw_pile = cards.copy()
            np.random.shuffle(self.draw_pile)
        else:
            print(f"Remaining cards: {len(self.draw_pile)}")
        return self.draw_pile.pop()

In [5]:
# Defining class to contain game state
class Game_State:
    # Instantiating base state
    def __init__(self, players):
        self.win_score = 10_000
        self.players_list = []
        for i in range(players):
            self.players_list.append(Player(f"Player {i+1}"))
        self.deck = Deck()
                                     
    # Method to quit the game
    def quit_game(self):
        #if input("Are you sure you want to quit? (y/n)").lower().strip() == "y":
        sys.exit("Thank you for playing!")

In [6]:
# Defining general function to get player input
def get_input(player, options={"0": "quit"}):
    options_guide = {"quit": player.quit_game, "draw": player.new_card, "roll dice": player.roll_dice, "save dice": player.save_dice, "end turn": player.end_turn}
    print("What would you like to do?")
    time.sleep(2)
    while True:
        action = input(f"Choose one of the following options: {options}").lower().strip()
        if (action in options.values()) or (action in options.keys()):
            for option in options.items():
                if action in option:
                    return options_guide[option[1]]()
            break
        else:
            print("Not a valid option - try entering the number or string for one of the options provided")

In [7]:
# Options guide is the index for the functions
#options_guide = {"quit": quit_game, "draw": new_card, "roll dice": roll_dice, "save dice": save_dice, "end turn": end_turn}


In [8]:
# Collecting the number of players from the user
while True:
    try:
        players = int(input("How many players? Enter a number: "))
    except:
        print("Not a valid number of players - please enter an integer")
        continue
    if (players > 1):
        break
    else:
        print("You can't play with fewer than two people!")
        continue

How many players? Enter a number:  2


In [9]:
# Turn loop - this contains the actual gameplay
#game = Game_State(players=players)
deck = Deck()
players_list = []
for i in range(players):
            players_list.append(Player(f"Player {i+1}"))
while True:
    for player in players_list:
        print(f"{player.name}'s turn")
        player.turn_score = 0
        player.new_card(deck)
        if player.card == "NO DICE":
            print("Sorry, no dice!")
            continue
        #while set(player.dice) == {0}:
        if not get_input(player, {"1": "roll dice", "0": "quit"}):
            continue      
        get_input(player, {"1": "save dice", "0": "quit"})
    break #temporary one-turn limit until I figure out why quit isn't working

Player 1's turn
Remaining cards: 54
What would you like to do?


Choose one of the following options: {'1': 'roll dice', '0': 'quit'} 1


[4, 2, 4, 2, 2, 5]
What would you like to do?


Choose one of the following options: {'1': 'save dice', '0': 'quit'} 4


Not a valid option - try entering the number or string for one of the options provided


...

Which dice would you like to save? 1, 1, 1


Player 2's turn
Remaining cards: 53
What would you like to do?


Choose one of the following options: {'1': 'roll dice', '0': 'quit'} 0


SystemExit: Thank you for playing!

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
